## Import Python libraries

In [ ]:
import json
import logging
import pandas as pd 

from tqdm import tqdm
from pathlib import Path

from AgentFactory import AgentFactory
from utils import clear_json

logging.basicConfig(level=logging.WARNING)

/home/youyuan/miniconda3/envs/EMO_CTRL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load an agent with specific configurations

In [ ]:
# config_path: str = "configs/conf_LLaVA.json"
# config_path: str = "configs/conf_LLaMA3.json"
# config_path: str = "configs/conf_DeepSeek.json"
config_path: str = "configs/conf_GPT.json"
config = json.load(open(config_path, "r"))
config = config["model"]

agent_factory = AgentFactory()
agent = agent_factory.create_agent(config)

## Load the IPIP test data

In [ ]:
multi_list = pd.read_excel('IPIP/EN.xlsx')
instuments = multi_list['instrument'].unique()

## Let's run the agent with the IPIP test data

In [13]:
from Questionnaire.IPIPtest import IPIPTest

ProgressBar = tqdm(total=len(instuments), desc="Processing all instuments")

for instrument in tqdm(instuments):
    ProgressBar.update(1)
    print(f"Processing instrument: {instrument}")
    
    save_path = f"output/{config['template_type']}/{instrument}.jsonl"
    if Path(save_path).exists():
        print(f"File {save_path} already exists, skipping.")
        continue

    qn = IPIPTest(multi_list, instrument)

    for no, q in enumerate(qn.questions):
        
        agent.reset_msg_history()

        agent.instruction = config["system_msg"]
        prompt = [
            {"type": "text", "text": q},
        ]
        response, text = agent.get_response(prompt)

        try:
            dict = json.loads(clear_json(text))
            qn.record_answer(no, dict)
        except Exception as e:
            print(f"Error: {e}")
            print(f"text: {text}")
            continue

    # final_output = qn.analyze()
    qn.save_to_jsonl(f"output/{config['template_type']}/{instrument}.jsonl")
    print(agent.llm.total_cost())

Processing all instuments: 100%|██████████| 23/23 [00:09<00:00,  2.47it/s]


Processing instrument: 16PF
File output/GPT/16PF.jsonl already exists, skipping.
Processing instrument: 6FPQ


Processing all instuments:   8%|▊         | 3/36 [06:52<1:15:37, 137.49s/it]

0.002965
Processing instrument: 7FACTOR
File output/GPT/7FACTOR.jsonl already exists, skipping.
Processing instrument: AB5C
File output/GPT/AB5C.jsonl already exists, skipping.
Processing instrument: Barchard2001
File output/GPT/Barchard2001.jsonl already exists, skipping.
Processing instrument: BFAS
File output/GPT/BFAS.jsonl already exists, skipping.
Processing instrument: BFAS-20
File output/GPT/BFAS-20.jsonl already exists, skipping.
Processing instrument: BIDR
File output/GPT/BIDR.jsonl already exists, skipping.
Processing instrument: BIS_BAS


Processing all instuments:  28%|██▊       | 10/36 [08:20<17:57, 41.44s/it]  

...

0.0019150000000000003
Processing instrument: NEO5-20
File output/GPT/NEO5-20.jsonl already exists, skipping.
Processing instrument: ORAIS


Processing all instuments:  81%|████████  | 29/36 [57:00<20:39, 177.11s/it]

0.0015300000000000001
Processing instrument: ORVIS
File output/GPT/ORVIS.jsonl already exists, skipping.
Processing instrument: Radloff1977
File output/GPT/Radloff1977.jsonl already exists, skipping.
Processing instrument: Rosenberg1965
File output/GPT/Rosenberg1965.jsonl already exists, skipping.
Processing instrument: Scheier1994
File output/GPT/Scheier1994.jsonl already exists, skipping.
Processing instrument: Snyder1974
File output/GPT/Snyder1974.jsonl already exists, skipping.
Processing instrument: Span2002
File output/GPT/Span2002.jsonl already exists, skipping.
Processing instrument: TCI


100%|██████████| 36/36 [1:04:32<00:00, 107.58s/it][1:04:32<00:00, 102.74s/it]

0.00203
Processing instrument: VIA
File output/GPT/VIA.jsonl already exists, skipping.


##  print the cost of the agent

In [5]:
print(agent.llm.total_cost())

0.001335
